This tutorial can be downloaded [link](http://greatfire.uchicago.edu/west-public/West/raw/master/Doc/tutorials/advanced/advanced_001.ipynb).

# Advanced tutorial 1: Symmetry analysis in Quantum Defect Embedding Theory (QDET)

We explain how to identify the symmetry of excited many-body states calculated with the Quantum Defect Embedding Theory (QDET) using the point symmetry of the defect. 
In this tutorial, we focus on the $\mathrm{NV^-}$ center in diamond.

## Step 1: Mean-field starting point

As for a standard QDET calculation, we need to determine the mean-field starting point first. In order to perform the symmetry analysis, we will additionally write the single-particle wavefunctions to cube files, in order to determine their symmetry character.

Download the following files in your working directory:

In [ ]:
%%bash
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63/pw.in
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63/pp.in
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/C_ONCV_PBE-1.0.upf
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/N_ONCV_PBE-1.0.upf

We can now inspect the `pw.in` file, the input for the `pw.x` code:

In [ ]:
%%bash
cat pw.in

We can now run `pw.x` on 2 cores.

In [ ]:
%%bash
mpirun -n 2 pw.x -i pw.in > pw.out

Once the ground state calculation has finished successfully, we can write the single-particle wavefunctions using the `pp.x` executable. The input file is given as

In [ ]:
%%bash
cat pp.in

And the calculation is performed with the following command

In [ ]:
%%bash
mpirun -n 2 pp.x -i pp.in > pp.out

## Step 2: Definition of the point group

We now define the point group. A point group is characterized by two quantities:
* a set of symmetry operations (identity, reflection, and rotation operations)
* a corresponding character table

In [ ]:
import numpy as np
from westpy.qdet.symm import PointGroup, PointGroupOperation, PointGroupRotation, PointGroupReflection

sq3 = np.sqrt(3)

# the origin is set to the Nitrogen site.
# The prefactor corresponds to the real-space grid along each direction.
origin = 64 * np.array([0.48731,  0.48731,  0.48731])

point_group = PointGroup(
    name="C3v",
    operations={
        "E": PointGroupOperation(T=np.eye(4)),
        "C3_1": PointGroupRotation(rotvec=2 * np.pi / 3 * np.array([1/sq3, 1/sq3, 1/sq3]),
                                   origin=origin),
        "C3_2": PointGroupRotation(rotvec=4 * np.pi / 3 * np.array([1/sq3, 1/sq3, 1/sq3]),
                                   origin=origin),
        "Cv_1": PointGroupReflection(normal=(1, -1, 0), origin=origin),
        "Cv_2": PointGroupReflection(normal=(0, -1, 1), origin=origin),
        "Cv_3": PointGroupReflection(normal=(-1, 0, 1), origin=origin)
    },
    ctable={
        "A1": [1, 1, 1, 1, 1, 1],
        "A2": [1, 1, 1, -1, -1, -1],
        "E": [2, -1, -1, 0, 0, 0]})

## Step 3: Diagonalization of QDET Hamiltonian

In this tutorial, we focus on the symmetry analysis with QDET. To avoid costly calculations of the matrix elements of the effective Hamiltonian, we simply download the input and output files of the QDET calculation. To learn how to perform QDET calculations, see [Basic Tutorial 5](http://greatfire.uchicago.edu/west-public/West/raw/master/Doc/tutorials/basic/basic_005.ipynb).

In [ ]:
%%bash
mkdir -p west.wfreq.save
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63/wfreq.in
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63/wfreq.json -O west.wfreq.save/wfreq.json

To add point-group analysis to QDET, we simply pass the point-group object defined above to the Hamiltonian object. Additionally, we provide the list of filenames for the wavefunctions.

The code will apply the symmetry operations to the Kohn-Sham wavefunctions stored in the cube files and determine the character of these wavefunctions. After the diagonalization of the effective Hamiltonian, the character of the many-body states can then be inferred from the character of the wavefunctions.

In [ ]:
# Here, we provide the list of file paths for the cube files for each basis function of the active space.
wfct_filenames = ['wfct_K001_B087.cube','wfct_K001_B122.cube', 'wfct_K001_B123.cube',
                  'wfct_K001_B126.cube', 'wfct_K001_B127.cube', 'wfct_K001_B128.cube']

In [ ]:
# the diagonalization is exactly the same as without the symmetry analysis.
from westpy.qdet import QDETResult

# construct object for effective Hamiltonian
effective_hamiltonian = QDETResult(filename='west.wfreq.save/wfreq.json', point_group=point_group,
                                  wfct_filenames=wfct_filenames)

# diagonalize Hamiltonian
solution = effective_hamiltonian.solve()